# Impact of Central Bank Policies on Bond Markets and Yield Curves (2020–2025)

**Project Overview**  
This notebook analyses how changes in policy rates and yield curve spreads affect US Aggregate (AGG) and UK Gilts (IGLT.L) bond ETF returns. It then simulates hypothetical future bond market paths under different central bank policy scenarios starting from the most recent data (Dec 30, 2025).

**Data Sources**  
- FRED: 10Y-2Y Treasury spread (T10Y2Y), UK 10Y gilt yield (IRLTLT01GBM156N), UK 3M interbank rate (IR3TIB01GBM156N), Federal Funds Rate (FEDFUNDS)  
- Yahoo Finance: iShares Core US Aggregate Bond ETF (AGG), iShares Core UK Gilts ETF (IGLT.L)

**Methodology**  
1. Feature engineering (bond returns, spread changes, policy rate changes)  
2. OLS regression: Bond_Return ~ spread_change + policy_rate_change  
3. Visualisation of historical spreads and cumulative returns  
4. Simulation of future N-months paths under hypothetical policy scenarios

In [1]:
import sys
from pathlib import Path

repo_root = Path().resolve().parent
sys.path.insert(0, str(repo_root))

In [2]:
from src.policy_impacts.data_loader import DataLoader
from src.policy_impacts.data_analysis import DataAnalysis
from src.policy_impacts.simulate_events import EventSimulator

## 1. Data Loading & Historical Analysis

In [3]:
data_loader = DataLoader()
data_loader.load_data()

In [4]:
data_path = repo_root / "data/policy_impacts/combined_data.csv"
analysis = DataAnalysis(data_path)
analysis.visualise_yield_curves()
_, _, _ = analysis.market_analysis()
analysis.compare_bond_reactions()

## 2. Scenario Simulation: Hypothetical Policy Paths

### Scenario 1: 25 bp Fed Rate Hike (hawkish surprise)

In [5]:
simulator = EventSimulator(data_path)
simulator.fit_model()

hike_sim = simulator.simulate_event(
    name="Hawkish Surprise – 50 bp Fed Hike",
    total_us_rate_change=0.50,
    total_uk_rate_change=0.25,
    months=12,
    spread_impact_factor=0.6,
)
simulator.plot_simulation(hike_sim, title="Simulation: 50 bp Fed Rate Hike")


Scenario: Hawkish Surprise – 50 bp Fed Hike
Total US rate change: 50.0 bp
Total UK rate change: 25.0 bp
Over 12 months
Final cum US bond return (approx): -0.61%
Final cum UK bond return (approx): -0.89%
US spread change: 30.0 bp
UK spread change: 15.0 bp


### Scenario 2: 25 bp BoE Rate Cut (dovish / recession response)

In [6]:
cut_sim = simulator.simulate_event(
    name="Dovish / QE – 50 bp Cuts",
    total_us_rate_change=-0.50,
    total_uk_rate_change=-0.30,
    months=12,
    spread_impact_factor=0.0,
)
simulator.plot_simulation(cut_sim, title="Simulation: 50 bp Rate Cuts / QE")


Scenario: Dovish / QE – 50 bp Cuts
Total US rate change: -50.0 bp
Total UK rate change: -30.0 bp
Over 12 months
Final cum US bond return (approx): 0.01%
Final cum UK bond return (approx): -0.29%
US spread change: 0.0 bp
UK spread change: 0.0 bp


### Scenario 3: No Policy Change (status quo / hold)

In [7]:
base_sim = simulator.simulate_event(
    name="Policy Hold – No Rate Change",
    total_us_rate_change=0.0,
    total_uk_rate_change=0.0,
    months=12,
    spread_impact_factor=0.0
)
simulator.plot_simulation(base_sim, title="Simulation: No Policy Change")


Scenario: Policy Hold – No Rate Change
Total US rate change: 0.0 bp
Total UK rate change: 0.0 bp
Over 12 months
Final cum US bond return (approx): 0.06%
Final cum UK bond return (approx): -0.11%
US spread change: 0.0 bp
UK spread change: 0.0 bp


## 3. Key Takeaways & Limitations

- Bond returns are more sensitive to domestic policy rate changes than foreign ones  
- UK gilts (IGLT.L) tend to show higher volatility around BoE-related moves  
- Cumulative return paths diverge noticeably under opposing policy shocks  
- Limitations: simple linear projection, no volatility clustering, assumes constant coefficients, no forward-guidance / QE effects modelled explicitly